In [2]:
# ====================================================
# まずシンプルなRAGを実装
# ====================================================
# document_loadersを使ってGitHubのリポジトリからドキュメントをロード
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))


387


In [3]:
# Embedding --> Chromaにインデクシング
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)


In [19]:
# シンプルなRAGを実装
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''
    以下の文脈だけを踏まえて質問に回答してください。
    文脈："""
    {context}
    """

    質問：
    {question}
    ''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
retriever = db.as_retriever()

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("LangChainの概要を教えてください。")
print(output)


LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としており、以下のような機能を提供しています。

1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。

2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。

3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。

LangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの主要コンポーネントに対して標準化されたインターフェースを提供し、開発者が異なるプロバイダー間で簡単に切り替えられるようにします。また、複雑なアプリケーションのためのオーケストレーション機能を提供し、アプリケーションの可視性と評価を向上させるためのツールも備えています。


## 検索クエリの工夫

In [ ]:
# ====================================================
# HyDE(Hypothetical Document Embeddings)
# 質問に回答する仮説的な回答を推論させ、その出力をドキュメントの検索に使用する
# ====================================================
hy_prompt = ChatPromptTemplate.from_template("""
    以下の質問に回答する一文を作成してください

    質問：{question}
    """)

hy_chain = hy_prompt | model | StrOutputParser()

hyde_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hy_chain | retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

output = hyde_rag_chain.invoke("LangChainの概要を教えてください。")
print(output)


In [ ]:
# ====================================================
# 複数の検索クエリの生成
# ====================================================
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(description="質問に回答するための複数の検索クエリ")


query_generation_prompt = ChatPromptTemplate.from_template("""\
    質問に対してベクターデータベースから関連文書を検索するために、
    3つの異なる検索クエリを生成してください。
    距離ベースの類似性検索の限界を克服するために、
    ユーザーの質問に対して複数の視点を提供することが目標です。

    質問：{question}
    """)

query_generation_chain = (
    query_generation_prompt | model.with_structured_output(QueryGenerationOutput) | (lambda x: x.queries)
)

multi_query_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map(),  # mapで引数と戻り値をlist化できる
    }
    | prompt
    | model
    | StrOutputParser()
)

output = multi_query_rag_chain.invoke("LangChainの概要を教えてください。")
print(output)


## 検索後の工夫

In [9]:
# ====================================================
# RAG-Fusion
# 複数の検索クエリを生成し、それらの検索結果をRRFで並べる手法
# RRF(Reciprocal Rank Fusion)
# ====================================================
from IPython.display import Image

print("RAG-Fusionのイメージ図")
Image(url="https://miro.medium.com/v2/resize:fit:893/1*EA6LTHgoZzc9RloYlKRIKw.png")


RAG-Fusionのイメージ図


In [18]:
# RRFの実装
from collections import defaultdict

from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[Document]:
    # Documentオブジェクトとスコアの対応を保持する辞書を準備
    doc_score_mapping = defaultdict(float)

    # 各Documentオブジェクトを保持する辞書
    content_to_doc = {}

    # 検索クエリ毎にループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs, start=1):
            content = doc.page_content
            # Documentオブジェクトを保存
            content_to_doc[content] = doc
            # スコアを加算
            doc_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(doc_score_mapping.items(), key=lambda x: x[1], reverse=True)
    # Documentオブジェクトのリストを返す
    return [content_to_doc[content] for content, _ in ranked]


rag_fusion_chain = (
    {
        "question": RunnablePassthrough(),
        "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
    }
    | prompt
    | model
    | StrOutputParser()
)

rag_fusion_chain.invoke("LangChainの概要を教えてください。")


'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としています。具体的には、以下のような機能を提供しています：\n\n- **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n- **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n- **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。これにより、開発者は異なるコンポーネントを簡単に組み合わせたり、プロバイダーを切り替えたりすることができます。'

In [17]:
# ====================================================
# リランクモデル
# ====================================================
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document


def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(
        model="rerank-multilingual-v3.0", top_n=top_n
    )  # top_n:リランク結果の上位何件を返すか

    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {"question": RunnablePassthrough(), "documents": retriever}
    | RunnablePassthrough.assign(context=rerank)
    | prompt
    | model
    | StrOutputParser()
)
rerank_rag_chain.invoke("LangChainの概要を教えてください。")


'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、チャットモデルや埋め込みモデル、ベクトルストアなどの関連技術に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、特に`langchain-core`、`langchain`、`langchain-community`、`langgraph`などのパッケージがあります。\n\nさらに、LangChainは、開発者がアプリケーションを構築する際に直面するさまざまな課題に対処するための標準化されたコンポーネントインターフェース、オーケストレーション機能、観測性と評価のサポートを提供しています。'

In [ ]:
## RAG-Fusion-->リランクする例

# rag_fusion_rerank_chain = (
#     {
#         "question": RunnablePassthrough(),
#         "documents": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
#     }
#     | RunnablePassthrough.assign(context=rerank)
#     | prompt
#     | model
#     | StrOutputParser()
# )
# rag_fusion_rerank_chain.invoke("LangChainの概要を教えてください。")


## 複数のRetrieverを使う工夫


In [26]:
# ====================================================
# LLMによるルーティング
# 質問内容に応じて適切なRetriever（web検索orドキュメント検索）を選択する
# ====================================================
from enum import Enum

from langchain_community.retrievers import TavilySearchAPIRetriever

# トレースがわかりやすくなるよう、run_nameを設定
langchain_document_retriever = retriever.with_config({"run_name": "langchain_document_retriever"})
web_retriever = TavilySearchAPIRetriever(k=3).with_config({"run_name": "web_retriever"})


# Retrieverの選択を行うチェーンを作成
class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
    質問に回答するために適切なRetrieverを選択してください。
    質問：{question}
    """)

route_chain = route_prompt | model.with_structured_output(RouteOutput) | (lambda x: x.route)


# ルーティングの結果を踏まえて検索するrouted_retriever関数と、処理全体の流れのChainを実装
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown retriever: {route}")


route_rag_chain = (
    {"question": RunnablePassthrough(), "route": route_chain}
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt
    | model
    | StrOutputParser()
)

# ドキュメント検索を選択する例
# route_rag_chain.invoke("LangChainの概要を教えてください。")

# Web検索を選択する例
route_rag_chain.invoke("2025年1月現在の日本の首相を教えてください")


'2025年1月現在の日本の首相は石破茂です。'

In [29]:
# ====================================================
# ハイブリッド検索
# 埋め込みベクトルでの類似度検索 x BM25を使った全文検索
# ====================================================
from langchain_community.retrievers import BM25Retriever
from langchain_core.runnables import RunnableParallel

# ドキュメント検索
chroma_retriever = retriever.with_config({"run_name": "chroma_retriever"})
# 全文検索
bm25_retriever = BM25Retriever.from_documents(documents).with_config({"run_name": "bm25_retriever"})

# 両方の検索を実施するChain
hybrid_retriever = (
    RunnableParallel(
        {
            "chroma_documents": chroma_retriever,
            "bm25_documents": bm25_retriever,
        }
    )
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt
    | model
    | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えてください。")


'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、アプリケーションのライフサイクルの各段階を簡素化することを目的としており、以下のような機能を提供しています。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなモデルや関連コンポーネントに対して標準化されたインターフェースを提供し、開発者がプロバイダー間で簡単に切り替えたり、コンポーネントを組み合わせたりできるようにします。また、複雑なアプリケーションの構築を支援するためのオーケストレーション機能や、アプリケーションの可視性と評価を向上させるためのツールも提供しています。'